<center><strong><h1>TUGAS III</h1></strong></center>
<p style="margin-left:33.3333%;font-size:20px;font-weight:600;color:blue;">PENGANTAR DATA MINING</p>
<p style="margin-left:40%;font-size:20px;font-weight:600">Nama Anggota:</p>
<ul style="margin-left:28%;font-size:20px;font-weight:600">
    <li>Maftuh Mashuri (11160940000076)</li>
    <li>Zahrotul Aulia (11160940000024)</li>
    <li>Alif Tito SA (11160940000052)</li>
    <li>Khairul Umam (11160940000073)</li>
    <li>Fathi Syuhada (11150940000001)</li>
</ul>


# Import modul

In [5]:
import json
import pandas as pd
import nltk
from nltk.tokenize import TweetTokenizer as tw_tokenizer
from unidecode import unidecode
import time
import math

# Pengoperasian file
<p>Membuka file dan memasukkan data tweet json dan menyimpannya kedalam variabel list <i>twetts_data</i></p>

In [20]:
kelompok1 = pd.read_csv('data/kelompok1.csv')
#kelompok2_1 = pd.read_csv('data/kelompok2_1.csv')
#kelompok2_2 = pd.read_csv('data/kelompok2_2.csv')
#kelompok2_3 = pd.read_csv('data/kelompok2_3.csv')
#kelompok3 = pd.read_csv('data/kelompok3.csv')
#kelompok4 = pd.read_csv('data/kelompok4.csv')
#kelompok5 = pd.read_csv('data/kelompok5.csv')

dataset = []
for tweet in kelompok1['text']:
    dataset.append(tweet)
# for tweet in kelompok2_1['text']:
#     dataset.append(tweet)
# for tweet in kelompok2_2['text']:
#     dataset.append(tweet)
# for tweet in kelompok2_3['text']:
#     dataset.append(tweet)
# # for tweet in kelompok3['text']:
# #     dataset.append(tweet)
# for tweet in kelompok4['text']:
#     dataset.append(tweet)

In [21]:
tweetsIn = pd.DataFrame({'text':dataset})
tweetsIn.head()

,text
0,@danangtrip whahaha iya sih bener
1,#Repost antv_official\r\n• • • • • •\r\nLedaka...
2,[00:16] #JAKARTA #MACET Kalideres arah Batu Ce...
3,[00:10] #JAKARTA Tol Cawang - TMII - Cibubur -...
4,"Well, makin kesini ku tahu yang mana yang mint..."


# Pengoperasian stopword
<ul>
    <li>Membuka file stopword(indonesia, inggris, noise)</li>
    <li>kemudian menggabungkan semua kata yang ada di ketiga file sehingga menjadi satu teks panjang dan menyimpannya kedalam variabel <i>stopword_file_all</i></li>
    <li>kemudian teks panjang di tokenize(dipisah perkata) dan menyimpannya ke dalam variabel list <i>stopwords</i></li>
</ul>

In [22]:
stopword_file1 = open('stopword/stopword_id.txt', "r").read() # Membuka file stopword bahasa indonesia dan menjadikan isi file tersebut sebagai string
stopword_file2 = open('stopword_en/stopwords_en.txt', "r").read()  # Membuka file stopword bahasa inggris dan menjadikan isi file tersebut sebagai string
stopword_file3 = open('stopword_noise/stopword_noise.txt', "r").read()  # Membuka file stopword noise dan menjadikan isi file tersebut sebagai string
stopword_file_all = stopword_file1 + stopword_file2 + stopword_file3  # Menggabungkan ketiga string stopword sebelumnya kedalam satu string
stopwords = stopword_file_all.split('\n') # Memisahkan kata dalam string yang sudah digambungkan berdasarkan baris
# print(stopwords)

# Pengoperasian slangwords
<ul>
    <li>Membuka file slangword(<i>colloquial-indonesian-lexicon.csv</i> dan <i>20190327_slangword.txt</i>)</li>
    <li>File yang pertama membuka dengan modul pandas dan mengkonfersinya kedalam dataframe, kemudian menyimpan masing-masing kata kedalam variabel dictionary <i>slangwords</i></li>
    <li>File yang kedua sama halnya pengoperasian pada file stopwords kemudian menyimpan masing-masing kata kedalam variabel dictionary <i>slangwords</i></li>
</ul>

In [23]:
slangwords = dict() # Membuat dictionary kosong untuk menyimpan kata slang dan formal sebagai key dan value
slangwords_dataframe = pd.read_csv('slangword/colloquial-indonesian-lexicon.csv') # Membuka file csv yang berisi kata slang dan formal dan mengkonversi kedalam dataframe
for slang, formal in zip(slangwords_dataframe['slang'], slangwords_dataframe['formal']):
    slangwords[slang] = formal # Mapping kata slang dan formal dan memasukkan ke dalam dictionary secara berulang

slangword_file = open('slangword/slangword.txt', "r").read() # Membuka file yang berisi kata slang dan kata formal dan mengkonversi kedalam string
slangwords_text = slangword_file.split('\n') # Memisahkan kata berdasarkan baris namun kata slang dan kata formal masih belum terpisah. output : (['slang:formal', ...])
#print(slangwords_text)
for slang in slangwords_text:
    split_slang = slang.split(":") # Memisahkan semua kata slang dan kata formal berdasarkan "titik dua (:)"
    slangwords[split_slang[0]] = split_slang[1] # Mapping semua kata slang dan kata formal ke dalam dictionary. Output : {'slang' : 'formal', ...}
#print(slangwords)

# 2-3. Tokenisasi dan Filtering 
<ul>
    <li>Mapping semua tweet dan menyimpannya kedalam variabel list <i>array_text</i></li>
    <li>Menggabungkan semua teks kedalam satu teks panjang <i>long_text</i></li>
    <li>Memisahkan teks perkata</li>
    <li>Menghapus simbol, ASCII, link (https: atau www.), dan kata lainnya</li>
</ul>

In [24]:
tweetsIn['tokenized'] = list(map(lambda tweet: tw_tokenizer(strip_handles=True, reduce_len=True).tokenize(unidecode(tweet).lower()), tweetsIn['text'])) # Memisahkan kata dalam text berasarkan "spasi"
tweetsIn['clean_text'] = list(map(lambda tweet: [w for w in tweet if w.isalnum()], tweetsIn['tokenized'])) # Filtering kata yang hanya berisi karakater a-z dan 0-9 (Menghapus url, hashtag, mention)
tweetsIn.head()

,text,tokenized,clean_text
0,@danangtrip whahaha iya sih bener,"[whahaha, iya, sih, bener]","[whahaha, iya, sih, bener]"
1,#Repost antv_official\r\n• • • • • •\r\nLedaka...,"[#repost, antv_official, *, *, *, *, *, *, led...","[ledakan, mercon, besar, menghancurkan, rumah,..."
2,[00:16] #JAKARTA #MACET Kalideres arah Batu Ce...,"[[, 00:16, ], #jakarta, #macet, kalideres, ara...","[kalideres, arah, batu, ceper, tangerang]"
3,[00:10] #JAKARTA Tol Cawang - TMII - Cibubur -...,"[[, 00:10, ], #jakarta, tol, cawang, -, tmii, ...","[tol, cawang, tmii, cibubur, bogor, ciawi]"
4,"Well, makin kesini ku tahu yang mana yang mint...","[well, ,, makin, kesini, ku, tahu, yang, mana,...","[well, makin, kesini, ku, tahu, yang, mana, ya..."


# 5. Handling SlangWord
<p>Mengubah kata-kata yang sudah melalui tahap <i>Stopword removal</i> dari kata slang menjadi kata formal</p>

In [25]:
tweetsIn['handled_slangword'] = list(map(lambda tweet : list(map(lambda w : slangwords[w] if w in slangwords.keys() else w, tweet)), tweetsIn['clean_text']))
# Mengubah kata slang menjadi kata formal (kata slang dan kata formal yang diperoleh dari dictionary) 
#print(handled_slangword[:100])
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword
0,@danangtrip whahaha iya sih bener,"[whahaha, iya, sih, bener]","[whahaha, iya, sih, bener]","[whahaha, iya, sih, benar]"
1,#Repost antv_official\r\n• • • • • •\r\nLedaka...,"[#repost, antv_official, *, *, *, *, *, *, led...","[ledakan, mercon, besar, menghancurkan, rumah,...","[ledakan, mercon, besar, menghancurkan, rumah,..."
2,[00:16] #JAKARTA #MACET Kalideres arah Batu Ce...,"[[, 00:16, ], #jakarta, #macet, kalideres, ara...","[kalideres, arah, batu, ceper, tangerang]","[kalideres, arah, batu, pendek, tangerang]"
3,[00:10] #JAKARTA Tol Cawang - TMII - Cibubur -...,"[[, 00:10, ], #jakarta, tol, cawang, -, tmii, ...","[tol, cawang, tmii, cibubur, bogor, ciawi]","[tol, cawang, tmii, cibubur, bogor, ciawi]"
4,"Well, makin kesini ku tahu yang mana yang mint...","[well, ,, makin, kesini, ku, tahu, yang, mana,...","[well, makin, kesini, ku, tahu, yang, mana, ya...","[well, makin, kesini, ku, tahu, yang, mana, ya..."
5,Honestly setelah balik ke indonesia hal yg pal...,"[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,..."
6,@detikcom Kang boong di bohongin gmn rasanya??,"[kang, boong, di, bohongin, gmn, rasanya, ?, ?]","[kang, boong, di, bohongin, gmn, rasanya]","[kang, bohong, di, bohongin, bagaimana, rasanya]"
7,Dikecewain berkali-kali tapi tetep cinta sama ...,"[dikecewain, berkali-kali, tapi, tetep, cinta,...","[dikecewain, tapi, tetep, cinta, sama, hehe, d...","[dikecewain, tapi, tetap, cinta, sama, hehe, d..."
8,@AbdillahMakmur @9GAG hee aku jadi inget mingg...,"[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, ingat, minggu, kemarin, tidak..."
9,Lah donnaruma ngapa itu udh diganti aja,"[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, mengapa, itu, sudah, diganti,..."


# 4. Stopword removal
<p>Menghapus kata-kata yang sering muncul dan tidak memiliki makna (yang, di, kan)</p>

In [26]:
tweetsIn['removed_stopwords'] = list(map(lambda tweet : [w for w in tweet if w not in stopwords], tweetsIn['handled_slangword']))
# Filtering data dengan menghapus kata yang tidak bermakna (Stopword yang diperoleh dari file)
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword,removed_stopwords
0,@danangtrip whahaha iya sih bener,"[whahaha, iya, sih, bener]","[whahaha, iya, sih, bener]","[whahaha, iya, sih, benar]",[whahaha]
1,#Repost antv_official\r\n• • • • • •\r\nLedaka...,"[#repost, antv_official, *, *, *, *, *, *, led...","[ledakan, mercon, besar, menghancurkan, rumah,...","[ledakan, mercon, besar, menghancurkan, rumah,...","[ledakan, mercon, menghancurkan, rumah, nanang..."
2,[00:16] #JAKARTA #MACET Kalideres arah Batu Ce...,"[[, 00:16, ], #jakarta, #macet, kalideres, ara...","[kalideres, arah, batu, ceper, tangerang]","[kalideres, arah, batu, pendek, tangerang]","[kalideres, batu, pendek, tangerang]"
3,[00:10] #JAKARTA Tol Cawang - TMII - Cibubur -...,"[[, 00:10, ], #jakarta, tol, cawang, -, tmii, ...","[tol, cawang, tmii, cibubur, bogor, ciawi]","[tol, cawang, tmii, cibubur, bogor, ciawi]","[tol, cawang, tmii, bogor, ciawi]"
4,"Well, makin kesini ku tahu yang mana yang mint...","[well, ,, makin, kesini, ku, tahu, yang, mana,...","[well, makin, kesini, ku, tahu, yang, mana, ya...","[well, makin, kesini, ku, tahu, yang, mana, ya...","[kesini, ku, tolong, manfaatin]"
5,Honestly setelah balik ke indonesia hal yg pal...,"[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, berasa, beda, cuaca, astagah, kerin..."
6,@detikcom Kang boong di bohongin gmn rasanya??,"[kang, boong, di, bohongin, gmn, rasanya, ?, ?]","[kang, boong, di, bohongin, gmn, rasanya]","[kang, bohong, di, bohongin, bagaimana, rasanya]","[kang, bohong, bohongin]"
7,Dikecewain berkali-kali tapi tetep cinta sama ...,"[dikecewain, berkali-kali, tapi, tetep, cinta,...","[dikecewain, tapi, tetep, cinta, sama, hehe, d...","[dikecewain, tapi, tetap, cinta, sama, hehe, d...","[dikecewain, cinta, dasar]"
8,@AbdillahMakmur @9GAG hee aku jadi inget mingg...,"[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, ingat, minggu, kemarin, tidak...","[hee, kemarin, sengaja, habis, menginjak, koch..."
9,Lah donnaruma ngapa itu udh diganti aja,"[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, mengapa, itu, sudah, diganti,...","[donnaruma, diganti]"


# 6. Lemmalization
<p>mengubah kata-kata dalam dataset tweet menjadi kata dasar dengan menggunakan modul <i>spacy</i> dan <i>sastrawi</i></p>

# Spacy

In [ ]:
from spacy.lang.id import Indonesian # Import modul spacy bahasa indonesia

nlp = Indonesian() # memanggi objek Indonesian() pada modul spacy
def stem_spacy(text): # Fungsi untuk mengubah kata-kata menjadi kata dasar
    for txt in nlp(text):
        t = txt.lemma_
    return t
tweetsIn['stemmed_by_spacy'] = list(map(lambda tweet : list(map(lambda word: stem_spacy(word), tweet)), tweetsIn['removed_stopwords']))
#tweetsIn.to_csv("data/" + str(time.time()) + "_export_clean_text.csv")
tweetsIn.head(10)

,text,tokenized,clean_text,handled_slangword,removed_stopwords,stemmed_by_spacy
0,@danangtrip whahaha iya sih bener,"[whahaha, iya, sih, bener]","[whahaha, iya, sih, bener]","[whahaha, iya, sih, benar]",[whahaha],[whahaha]
1,#Repost antv_official\r\n• • • • • •\r\nLedaka...,"[#repost, antv_official, *, *, *, *, *, *, led...","[ledakan, mercon, besar, menghancurkan, rumah,...","[ledakan, mercon, besar, menghancurkan, rumah,...","[ledakan, mercon, menghancurkan, rumah, nanang...","[ledak, mercon, hancur, rumah, nanang, tewas, ..."
2,[00:16] #JAKARTA #MACET Kalideres arah Batu Ce...,"[[, 00:16, ], #jakarta, #macet, kalideres, ara...","[kalideres, arah, batu, ceper, tangerang]","[kalideres, arah, batu, pendek, tangerang]","[kalideres, batu, pendek, tangerang]","[kalideres, batu, pendek, tangerang]"
3,[00:10] #JAKARTA Tol Cawang - TMII - Cibubur -...,"[[, 00:10, ], #jakarta, tol, cawang, -, tmii, ...","[tol, cawang, tmii, cibubur, bogor, ciawi]","[tol, cawang, tmii, cibubur, bogor, ciawi]","[tol, cawang, tmii, bogor, ciawi]","[tol, cawang, tmii, bogor, ciawi]"
4,"Well, makin kesini ku tahu yang mana yang mint...","[well, ,, makin, kesini, ku, tahu, yang, mana,...","[well, makin, kesini, ku, tahu, yang, mana, ya...","[well, makin, kesini, ku, tahu, yang, mana, ya...","[kesini, ku, tolong, manfaatin]","[kesini, ku, tolong, manfaatin]"
5,Honestly setelah balik ke indonesia hal yg pal...,"[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, setelah, balik, ke, indonesia, hal,...","[honestly, berasa, beda, cuaca, astagah, kerin...","[honestly, rasa, beda, cuaca, astagah, keringeta]"
6,@detikcom Kang boong di bohongin gmn rasanya??,"[kang, boong, di, bohongin, gmn, rasanya, ?, ?]","[kang, boong, di, bohongin, gmn, rasanya]","[kang, bohong, di, bohongin, bagaimana, rasanya]","[kang, bohong, bohongin]","[kang, bohong, bohongin]"
7,Dikecewain berkali-kali tapi tetep cinta sama ...,"[dikecewain, berkali-kali, tapi, tetep, cinta,...","[dikecewain, tapi, tetep, cinta, sama, hehe, d...","[dikecewain, tapi, tetap, cinta, sama, hehe, d...","[dikecewain, cinta, dasar]","[dikecewain, cinta, dasar]"
8,@AbdillahMakmur @9GAG hee aku jadi inget mingg...,"[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, inget, minggu, kemaren, ga, s...","[hee, aku, jadi, ingat, minggu, kemarin, tidak...","[hee, kemarin, sengaja, habis, menginjak, koch...","[hee, kemarin, sengaja, habis, injak, kocheng,..."
9,Lah donnaruma ngapa itu udh diganti aja,"[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, ngapa, itu, udh, diganti, aja]","[lah, donnaruma, mengapa, itu, sudah, diganti,...","[donnaruma, diganti]","[donnaruma, diganti]"


# Konfigurasi

In [ ]:
tweetsInreset = tweetsIn.reset_index()
DATA = tweetsInreset['stemmed_by_spacy']
N = 5 # Banyaknya

# Membuat list term atau kumpulan kata

In [ ]:
def create_term(text_list, total_text = 0):
    wordcloud = []
    if total_text == 0:
        total_text = len(text_list)

    for word in text_list[:total_text]:
        wordcloud += word

    term = list(dict.fromkeys(wordcloud))
    return term

print(create_term(DATA))

['whahaha', 'ledak', 'mercon', 'hancur', 'rumah', 'nanang', 'tewas', 'asih', 'bantu', 'kalideres', 'batu', 'pendek', 'tangerang', 'tol', 'cawang', 'tmii', 'bogor', 'ciawi', 'kesini', 'ku', 'tolong', 'manfaatin', 'honestly', 'rasa', 'beda', 'cuaca', 'astagah', 'keringeta', 'kang', 'bohong', 'bohongin', 'dikecewain', 'cinta', 'dasar', 'hee', 'kemarin', 'sengaja', 'habis', 'injak', 'kocheng', 'allah', 'kochengnya', 'donnaruma', 'diganti', 'balas', 'tidur', 'dibilang', 'mulu', 'dikasianin', 'bucin', 'banget', 'opo', 'iso', 'kecuali', 'siji', 'tangi', 'esuk', 'dikasih', 'ngember', 'cerita', 'alesannya', 'simpl', 'sepotong', 'kronologinya', 'liput', 'kal', 'seakan', 'atmosfir', 'today', 'ubah', 'oaoe', 'sweet', 'menuaa', 'tampung', 'aspirasi', 'warga', 'butuh', 'refresh', 'sejuk', 'hibur', 'tenang', 'malas', 'cari', 'angkat', 'tangan', 'masak', 'indomie', '62', 'negara', 'kemarau', 'penghujan', 'jas', 'hujan', 'seru', 'umbrella', 'academy', 'yaelah', 'kemana', 'corona', 'untung', 'fast', 'fu

# Membuat TF

In [ ]:
def create_tf(text_list, total_text= 0):
    if total_text == 0:
        total_text = len(text_list)
    term = create_term(text_list, total_text)
    tf = dict()
    for i in range(total_text):
        name = 'tf_text_' + str(i+1)
        tf[name] = list(map(lambda word: text_list[i].count(word), term))
    return tf

tf = create_tf(DATA)
# tf = create_tf(DATA, N)

pd.DataFrame(tf)

# Membuat IDF

In [ ]:
def create_idf(text_list, total_text = 0):
    if total_text == 0:
        total_text = len(text_list)
    idf = []
    term = create_term(text_list, total_text)
    tf = create_tf(text_list, total_text)
    total_words = len(term)

    for i in range(total_words):
        val = 0
        for j in range(total_text):
            name = 'tf_text_' + str(j+1)
            if float(tf[name][i]) > 0:
                val += 1
        idf.append(math.log10(total_text / float(val)))
    return idf

idf = create_idf(DATA)
# tf = create_idf(DATA, N)
idf

# Membuat TF-IDF

In [ ]:
def create_tf_idf(text_list, total_text = 0):
    if total_text == 0:
        total_text = len(text_list)
        
    tf_idf = dict()
    term = create_term(text_list, total_text)
    tf = create_tf(text_list, total_text)
    idf = create_idf(text_list, total_text)

    total_words = len(term)

    for i in range(total_text):
        name_tf = 'tf_text_' + str(i+1)
        name_tf_idf = 'tf_idf_text_' + str(i+1)
        list_tf_idf = []
        for j in  range(total_words):
            temp = 0
            temp = tf[name_tf][j] * idf[j]
            list_tf_idf.append(temp)
        tf_idf[name_tf_idf] = list_tf_idf
        
    return tf_idf

tf_idf = create_tf_idf(DATA)
# tf = create_tf_idf(DATA, N)

pd.DataFrame(tf_idf)

# TF, IDF, dan TF-IDF dikumpukan ke satu dataframe

In [ ]:
data = {**tf, 'idf' : idf, **tf_idf}
data_tf_idf = pd.DataFrame(data)
data_tf_idf